In [ ]:
pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os

In [ ]:
# gpus=tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import cv2
from PIL import Image
import shutil
from matplotlib import pyplot as plt

In [ ]:
og_data_dir='/kaggle/input/happy-and-sad-people'

In [ ]:
data_dir = '/kaggle/working/data_dir'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
image_exts=['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(og_data_dir):
    class_path = os.path.join(og_data_dir, image_class)
    
    if not os.path.isdir(class_path):
        continue
    
    target_class_dir = os.path.join(data_dir, image_class)
    os.makedirs(target_class_dir, exist_ok=True)

    for image in os.listdir(class_path):
        image_path = os.path.join(class_path, image)
        try:
            img = cv2.imread(image_path)
            with Image.open(image_path) as img_pil:
                img_format = img_pil.format.lower()
            
            if img_format in image_exts:
                # Copy valid image to the writable directory
                shutil.copy(image_path, os.path.join(target_class_dir, image))
            else:
                print(f'Skipped invalid format: {image_path}')
        
        except Exception as e:
            print(f'Issue with image: {image_path}, Error: {e}')

In [ ]:
img = cv2.imread(os.path.join(data_dir, 'happyppl', 'genuinely-happy-people-dont-complain-about.png'))

In [ ]:
# plt.imshow(img)

In [ ]:
# tf.data.Dataset??

In [ ]:
import numpy as np

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/kaggle/working/data_dir')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
# batch

In [ ]:
# Class 1 = Sad People
# Class 0 = Happy People
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
scaled_iterator.next()[0].max()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input

In [ ]:
model = Sequential([
    Input(shape=(256, 256, 3)),                        
    Conv2D(16, (3, 3), strides=1, activation='relu'),
    MaxPooling2D(),

    Conv2D(32, (3, 3), strides=1, activation='relu'),
    MaxPooling2D(),

    Conv2D(16, (3, 3), strides=1, activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')                    
])

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir = '/kaggle/working/logdir'
os.makedirs(logdir, exist_ok=True)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback]) 

In [ ]:
# hist.history

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x, y = batch
    yhat = model.predict(x)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
img = cv2.imread('/kaggle/input/happy-and-sad-people/happy.jpg')
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
# np.expand_dims(resize, 0).shape #NN expects batch of images not one image

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
models = '/kaggle/working/models'
os.makedirs(models, exist_ok=True)
model.save(os.path.join(models, 'emotionModel.h5'))

In [ ]:
new_model = load_model(os.path.join(models, 'emotionModel.h5'))

In [ ]:
yhatnew= new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if yhatnew > 0.5:
    print(f'The person is sad')
else:
    print(f'The person is happy')